#### 웹에서 데이터 다운 받은 후에 전처리 진행   
- 파일 용량이 커서 linux 이용해서 전처리 진행 후 파일 불러옴

gzcat /Users/jeongmin/Downloads/goa_uniprot_all.gaf.gz | grep -v '^!' | grep 'GO:' | grep 'taxon:10090' > /Users/jeongmin/Downloads/goa_mouse_filtered.gaf

pv goa_uniprot_all.gaf.gz | gzip -d | grep -v '^!' | grep 'GO:' | grep 'taxon:10090' > goa_mouse_filtered.gaf

pv /Users/jeongmin/Downloads/goa_uniprot_all.gaf.gz | gzip -d | grep -v '^!' | grep 'GO:' | grep 'taxon:10090' > /Users/jeongmin/Downloads/goa_mouse_filtered.gaf


gzcat /Users/jeongmin/Downloads/idmapping.dat.gz | grep -P '\tEnsembl\t' | head

gzcat /Users/jeongmin/Downloads/idmapping.dat.gz | grep '\tEnsembl\tENSMUSG' > /Users/jeongmin/Downloads/ensembl_mouse_mapping.tsv

gzcat /path/to/MOUSE_10090_idmapping.dat.gz | grep '	Ensembl	ENSMUSG' > ensembl_mouse_mapping.tsv

#### 이 코드는 colab에서 Guided mission 1을 다시 수행한 것

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io

# === df_filtered.csv ===
df = pd.read_csv('read-counts.txt', sep='\t', comment='#', index_col=0) # /content/drive/MyDrive/binfo1-work

# 컬럼 이름 간소화 및 정수로 변환
df.rename(columns={
    'CLIP-35L33G.bam': 'CLIP_35L33G',
    'RNA-control.bam': 'RNA_control', # 이 컬럼을 사용하거나, 아래 RNA_siLuc을 사용 (논문상 RNA_siLuc이 RPF와 함께 사용)
    'RNA-siLin28a.bam': 'RNA_siLin28a',
    'RNA-siLuc.bam': 'RNA_siLuc',
    'RPF-siLin28a.bam': 'RPF_siLin28a',
    'RPF-siLuc.bam': 'RPF_siLuc'
}, inplace=True)

# 필요한 컬럼만 정수형으로 변환 (raw count이므로)
count_cols = ['CLIP_35L33G', 'RNA_control', 'RNA_siLin28a', 'RNA_siLuc', 'RPF_siLin28a', 'RPF_siLuc']
for col in count_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# --- 1. 라이브러리 전체 리드 수 계산 (시뮬레이션) ---
# 경고: 이 값들은 제공된 샘플 데이터만을 기준으로 계산되었으며, 실제 라이브러리의 전체 리드 수와는 다릅니다.
# 실제 분석에서는 각 BAM 파일에서 얻은 총 리드 수를 사용해야 합니다.
total_clip_35l33g_reads = df['CLIP_35L33G'].sum()
total_rna_siluc_reads = df['RNA_siLuc'].sum()
total_rna_silin28a_reads = df['RNA_siLin28a'].sum()
total_rpf_siluc_reads = df['RPF_siLuc'].sum()
total_rpf_silin28a_reads = df['RPF_siLin28a'].sum()

# log2 변환 시 0이 되는 것을 피하기 위한 작은 상수 (pseudocount 또는 epsilon)
epsilon = 1e-9 # 논문에서 명시적으로 +1을 사용했으므로, 여기서 +1 대신 epsilon을 사용하지 않고, +1만 적용

# --- 2. 데이터 필터링 ---
# RNA-seq (siLuc) 리드 수가 30 미만인 유전자 제외 [6]
# RPF (siLuc) 리드 수가 80 미만인 유전자 제외 [5]
df_filtered = df[(df['RNA_siLuc'] >= 30) & (df['RPF_siLuc'] >= 80)].copy()

df_filtered['Normalized_CLIP'] = (df_filtered['CLIP_35L33G'] + 1) / (total_clip_35l33g_reads + 1)
    df_filtered['Normalized_RNA_siLuc_for_CLIP'] = (df_filtered['RNA_siLuc'] + 1) / (total_rna_siluc_reads + 1)
    df_filtered['CLIP_Enrichment'] = np.log2(df_filtered['Normalized_CLIP'] / df_filtered['Normalized_RNA_siLuc_for_CLIP'])

    # --- 4. Ribosome Density Change (Y축) 계산 ---
    # 각 조건의 번역 효율 (TE_raw = RPF / RNA) 계산
    df_filtered['TE_raw_siLin28a'] = (df_filtered['RPF_siLin28a'] + 1) / (df_filtered['RNA_siLin28a'] + 1)
    df_filtered['TE_raw_siLuc'] = (df_filtered['RPF_siLuc'] + 1) / (df_filtered['RNA_siLuc'] + 1)

    # 전체 라이브러리 스케일 정규화 인자 (Global TE) 계산
    global_te_siLin28a = (total_rpf_silin28a_reads + 1) / (total_rna_silin28a_reads + 1)
    global_te_siLuc = (total_rpf_siluc_reads + 1) / (total_rna_siluc_reads + 1)

    # 정규화된 번역 효율 계산 (TE_norm = TE_raw / Global_TE)
    # 0으로 나누는 것을 방지하기 위해 global_te 값에도 1을 더합니다.
    df_filtered['TE_norm_siLin28a'] = df_filtered['TE_raw_siLin28a'] / (global_te_siLin28a + epsilon)
    df_filtered['TE_norm_siLuc'] = df_filtered['TE_raw_siLuc'] / (global_te_siLuc + epsilon)

    # 리보솜 밀도 변화 = log2(TE_norm_siLin28a / TE_norm_siLuc)
    # log2(0) 방지를 위해 작은 epsilon을 추가합니다.
    df_filtered['Ribosome_Density_Change'] = np.log2((df_filtered['TE_norm_siLin28a'] + epsilon) / (df_filtered['TE_norm_siLuc'] + epsilon))

여기부터 YOA - project  
아래의 코드는 local에서 터미널 python 기반으로 작성

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import mannwhitneyu
import statsmodels.stats.multitest as smm
from matplotlib.colors import LogNorm
from goatools.obo_parser import GODag

In [ ]:
# === 경로 설정 ===
df_path = "/Users/jeongmin/Downloads/df_filtered.csv"
idmap_path = "/Users/jeongmin/Downloads/ensembl_mouse_mapping.tsv"
goa_path = "/Users/jeongmin/Downloads/goa_mouse_filtered.gaf"
obo_path = "/Users/jeongmin/Downloads/go-basic.obo"

In [ ]:
# === Step 1: 데이터 불러오기 ===
df_filtered = pd.read_csv(df_path, index_col=0)
df_filtered.index = df_filtered.index.str.split('.').str[0]
gene_rdc = df_filtered["Ribosome_Density_Change"].dropna()
gene_clip = df_filtered["CLIP_Enrichment"].dropna()

In [ ]:
# === Step 2: Ensembl → UniProt 매핑 ===
ensembl_to_uniprot = {}
with open(idmap_path, "rt") as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) >= 3 and parts[1] == "Ensembl":
            ens_id = parts[2].split('.')[0]
            if ens_id in df_filtered.index:
                ensembl_to_uniprot[ens_id] = parts[0]

In [ ]:
print(f"총 매핑된 Ensembl ID 수: {len(ensembl_to_uniprot)}")

In [ ]:
# 총 매핑된 Ensembl ID 수: 7946

In [ ]:
# 저장 (옵션)
df_map = pd.DataFrame(list(ensembl_to_uniprot.items()), columns=["Ensembl_ID", "UniProt_ID"])
df_map.to_csv("/Users/jeongmin/Downloads/ensembl_to_uniprot.csv", index=False)

In [ ]:
# === Step 3: UniProt → GO term 매핑 ===
uniprot_to_go = defaultdict(set)

with open(goa_path, "rt") as f:
    for line in f:
        if line.startswith("!"):
            continue
        parts = line.strip().split('\t')
        if len(parts) >= 5:
            uniprot_to_go[parts[1]].add(parts[4])

In [ ]:
# 저장 (옵션)
rows = [
    {"UniProt_ID": uid, "GO_Terms": ";".join(sorted(go_set))}
    for uid, go_set in uniprot_to_go.items()
]

df_go_map = pd.DataFrame(rows)
df_go_map.to_csv("/Users/jeongmin/Downloads/uniprot_to_go.csv", index=False)

In [ ]:
# === Step 4: GO term → Ensembl ID 매핑
go_to_genes = defaultdict(set)

for ens_id in df_filtered.index:
    uniprot_id = ensembl_to_uniprot.get(ens_id)
    if uniprot_id:
        for go_term in uniprot_to_go.get(uniprot_id, []):
            go_to_genes[go_term].add(ens_id)

In [ ]:
print(f"GO term 개수: {len(go_to_genes)}")

In [ ]:
# GO term 개수: 9079

In [ ]:
for i, (go, genes) in enumerate(go_to_genes.items()):
    print(f"{go} → {len(genes)} genes")
    if i >= 9:
        break  # 상위 10개만 보기

In [ ]:
from scipy.stats import mannwhitneyu
import statsmodels.stats.multitest as smm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# === Step 5: Mann-Whitney U Test
results = []
clip_results = []
for go_term, genes in go_to_genes.items():
    if len(genes) < 5:
        continue
    rdc_in = gene_rdc.loc[gene_rdc.index.isin(genes)]
    rdc_out = gene_rdc.loc[~gene_rdc.index.isin(genes)]
    clip_in = gene_clip.loc[gene_clip.index.isin(genes)]
    clip_out = gene_clip.loc[~gene_clip.index.isin(genes)]
    if len(rdc_in) > 0 and len(rdc_out) > 0:
        _, pval_rdc = mannwhitneyu(rdc_in, rdc_out, alternative="two-sided", method='asymptotic')
        results.append((go_term, len(genes), rdc_in.mean(), pval_rdc))
    if len(clip_in) > 0 and len(clip_out) > 0:
        _, pval_clip = mannwhitneyu(clip_in, clip_out, alternative="two-sided", method='asymptotic')
        clip_results.append((go_term, pval_clip))

In [ ]:
for row in results[:5]:
    print(f"GO: {row[0]}, N={row[1]}, Mean RDC={row[2]:.3f}, p={row[3]:.4e}")

In [ ]:
# === Step 6: FDR 보정 및 병합
results_df = pd.DataFrame(results, columns=["GO_term", "n_genes", "mean_rdc", "pval"])
results_df["fdr"] = smm.multipletests(results_df["pval"], method="fdr_bh")[1]
clip_df = pd.DataFrame(clip_results, columns=["GO_term", "clip_pval"])
clip_df["clip_fdr"] = smm.multipletests(clip_df["clip_pval"], method="fdr_bh")[1]
results_df = results_df.merge(clip_df, on="GO_term", how="left")

In [ ]:
results_df.head

<bound method NDFrame.head of          GO_term  n_genes  mean_rdc  ...           fdr     clip_pval      clip_fdr
0     GO:0003676      265 -0.624921  ...  9.154610e-24  1.009489e-02  8.975362e-02
1     GO:0005634     1943 -0.448916  ...  1.132890e-60  2.864048e-12  5.969948e-10
2     GO:0008270      616 -0.421909  ...  3.467431e-12  2.758326e-01  5.817230e-01
3     GO:0006368       15 -0.749462  ...  1.793866e-02  2.920672e-02  1.750537e-01
4     GO:0006351       39 -0.371901  ...  4.102666e-01  2.945847e-01  6.037057e-01
...          ...      ...       ...  ...           ...           ...           ...
1871  GO:0006607        5 -0.577579  ...  3.267266e-01  1.545350e-01  4.303612e-01
1872  GO:0045275        6  0.395308  ...  9.125456e-02  5.619788e-03  6.059036e-02
1873  GO:0006122        7  0.357409  ...  6.856592e-02  2.772726e-03  3.796813e-02
1874  GO:0004693        5 -0.461616  ...  6.038793e-01  2.971295e-01  6.047519e-01
1875  GO:0045598        5 -0.426737  ...  9.037637e-01  3.257185e-01  6.317473e-01

In [ ]:
# === CLIP 평균 계산
clip_enrichment = {
    go_term: df_filtered.loc[df_filtered.index.isin(go_to_genes[go_term]), "CLIP_Enrichment"].dropna().mean()
    for go_term in results_df["GO_term"]
}
results_df["clip_enrichment"] = results_df["GO_term"].map(clip_enrichment)

In [ ]:
# === Step 7: FDR 기준 필터링
sig_results = results_df[(results_df["fdr"] < 0.05) & (results_df["clip_fdr"] < 0.05)].copy()

In [ ]:
sig_results.head

<bound method NDFrame.head of          GO_term  n_genes  mean_rdc          pval           fdr     clip_pval      clip_fdr  clip_enrichment
1     GO:0005634     1943 -0.448916  1.811658e-63  1.132890e-60  2.864048e-12  5.969948e-10        -0.793606
6     GO:0003677      532 -0.469376  7.019641e-19  7.746381e-17  1.775003e-03  3.027187e-02        -0.780596
13    GO:0005654      707 -0.442764  1.960801e-19  2.299040e-17  2.313112e-05  1.113562e-03        -0.801364
19    GO:0005783      460  0.200246  4.910549e-65  4.606095e-62  6.577667e-46  6.169852e-43         0.109618
27    GO:0005737     1771 -0.422874  3.185813e-41  9.960975e-39  3.404130e-17  1.064358e-14        -0.836202
...          ...      ...       ...           ...           ...           ...           ...              ...
1589  GO:0015293       23  0.422332  1.091545e-07  3.863657e-06  3.192605e-06  2.139045e-04         0.437023
1682  GO:0016125        7  0.461167  8.738155e-04  8.719563e-03  3.931384e-04  1.010312e-02         0.829972
1739  GO:0007156       13  0.357391  4.716005e-05  7.899309e-04  4.534547e-05  1.809960e-03         0.625480
1796  GO:0006865       16  0.613664  2.035179e-08  8.299991e-07  2.563551e-05  1.202305e-03         0.555117
1860  GO:0007160        7  0.612470  2.267674e-04  2.988157e-03  1.572019e-03  2.816384e-02         0.821609

[128 rows x 8 columns]>

In [ ]:
print(sig_results.sort_values("fdr").head(5))

 GO_term  n_genes  mean_rdc  ...            fdr  -log10(FDR)  clip_enrichment
90   GO:0016020     1789  0.153882  ...  2.889719e-237   236.539144        -0.128158
19   GO:0005783      460  0.200246  ...   4.606095e-62    61.336667         0.109618
1    GO:0005634     1943 -0.448916  ...   1.132890e-60    59.945812        -0.793606
70   GO:0005789      310  0.266367  ...   7.894996e-55    54.102648         0.166805
103  GO:0005886      669  0.042771  ...   6.211669e-39    38.206792        -0.134128

In [ ]:
# === Step 8: GO 이름 매핑 & subset term 제거 
go_dag = GODag(obo_path)
sig_go_terms = set(sig_results["GO_term"])
filtered_go_terms = set()

def is_subset_term(go1, go2):
    try:
        return go1 in go_dag[go2].get_all_children()
    except:
        return False

for go in sig_go_terms:
    if not any(is_subset_term(go, other) for other in sig_go_terms if go != other):
        filtered_go_terms.add(go)

filtered_sig_results = sig_results[sig_results["GO_term"].isin(filtered_go_terms)].copy()
filtered_sig_results["GO_name"] = filtered_sig_results["GO_term"].map(lambda go: go_dag[go].name if go in go_dag else go)

In [ ]:
def sci_notation_latex(x):
    """지수 표기: 1.3×10^{-108} 형태로 반환"""
    base, exp = f"{x:.1e}".split("e")
    return f"{base}×10$^{{{int(exp)}}}$"

filtered_sig_results["GO_label"] = filtered_sig_results.apply(
    lambda row: (
        f"{row['GO_name']} ({row['n_genes']})\n"
        f"C={sci_notation_latex(row['clip_fdr'])}, R={sci_notation_latex(row['fdr'])}"
    ),
    axis=1
)


In [ ]:
filtered_sig_results.head

<bound method NDFrame.head of          GO_term  ...                                           GO_label
0     GO:0005634  ...                nucleus (1943)\nC=6.0e-10 R=1.1e-60
2     GO:0003677  ...             DNA binding (532)\nC=3.0e-02 R=7.7e-17
14    GO:0005654  ...             nucleoplasm (707)\nC=1.1e-03 R=2.3e-17
16    GO:0005783  ...   endoplasmic reticulum (460)\nC=6.2e-43 R=4.6e-62
24    GO:0006281  ...              DNA repair (154)\nC=4.0e-02 R=2.6e-07
...          ...  ...                                                ...
1195  GO:0006465  ...  signal peptide processing (6)\nC=3.0e-02 R=3.9...
1263  GO:0007611  ...        learning or memory (8)\nC=3.7e-02 R=9.2e-03
1304  GO:0042552  ...              myelination (15)\nC=8.4e-03 R=1.7e-02
1504  GO:0140326  ...  ATPase-coupled intramembrane lipid transporter...
1796  GO:0006865  ...     amino acid transport (16)\nC=1.2e-03 R=8.3e-07

[73 rows x 10 columns]>

In [ ]:
filtered_sig_results.to_csv("/Users/jeongmin/Downloads/sig_results_GO_enrichment_filtered.csv", index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm

# === 저장된 파일 불러오기 ===
sig_results = pd.read_csv("/Users/jeongmin/Downloads/sig_results_GO_enrichment_filtered.csv")

In [ ]:
# 주석 달고 싶은 GO term 목록 (상위 FDR 낮은 10개 예시)
top_labels = filtered_sig_results.nsmallest(10, "clip_fdr").copy()
top_labels

         GO_term  ...                                           GO_label
87    GO:0016020  ...  membrane (1789)\nC=1.3×10$^{-108}$, R=2.9×10$^...
16    GO:0005783  ...  endoplasmic reticulum (460)\nC=6.2×10$^{-43}$,...
275   GO:0009986  ...  cell surface (85)\nC=6.7×10$^{-15}$, R=9.5×10$...
26    GO:0005737  ...  cytoplasm (1771)\nC=1.1×10$^{-14}$, R=1.0×10$^...
148   GO:0055085  ...  transmembrane transport (111)\nC=3.9×10$^{-10}...
338   GO:0005576  ...  extracellular region (184)\nC=5.5×10$^{-10}$, ...
0     GO:0005634  ...  nucleus (1943)\nC=6.0×10$^{-10}$, R=1.1×10$^{-...
33    GO:0005794  ...  Golgi apparatus (313)\nC=1.0×10$^{-9}$, R=1.7×...
341   GO:0007155  ...  cell adhesion (79)\nC=1.7×10$^{-9}$, R=4.7×10$...
1016  GO:0005788  ...  endoplasmic reticulum lumen (34)\nC=6.6×10$^{-...

In [ ]:

# 🔧 수동 위치 설정: {GO_term: (dx, dy)} → 주석의 xytext offset
arrow_offsets = {
    "GO:0016020": (-1.1, 0.4),   # membrane
    "GO:0005783": (1.0, 0.1),    # endoplasmic reticulum
    "GO:0009986": (0.2, 0.9),    # cell surface
    "GO:0005737": (0.9, -0.3),  # cytoplasm
    "GO:0055085": (0.05, 0.55),   # transmembrane transport
    "GO:0005576": (-0.9, 0.7),   # extracellular region
    "GO:0005634": (0.9, 0.05),    # nucleus
    "GO:0005794": (-1.2, 0.1),   # Golgi apparatus
    "GO:0007155": (0.8, -0.2),    # cell adhesion
    "GO:0005788": (0.9, 0.3),  # ER lumen
}


plt.figure(figsize=(12,6))
sc = plt.scatter(
    filtered_sig_results["clip_enrichment"], filtered_sig_results["mean_rdc"],
    s=filtered_sig_results["n_genes"] * 0.9, c=filtered_sig_results["clip_fdr"],
    norm=LogNorm(vmin=1e-20, vmax=1), cmap="YlOrRd_r", alpha=0.9, linewidths=0
)
# ✅ Top label들에만 검은색 테두리 추가
plt.scatter(
    top_labels["clip_enrichment"], top_labels["mean_rdc"],
    s=top_labels["n_genes"] * 0.9,
    facecolors='none', edgecolors='black', linewidths=0.4
)

cb = plt.colorbar(sc)
cb.set_label("Term-specific enrichment confidence (false discovery rate)")
cb.set_ticks([1e-20, 1e-15, 1e-10, 1e-5, 1e0])
cb.set_ticklabels(["10⁻²⁰", "10⁻¹⁵", "10⁻¹⁰", "10⁻⁵", "10⁰"])
cb.ax.invert_yaxis()

for _, row in top_labels.iterrows():
    dx, dy = arrow_offsets.get(row["GO_term"], (0.5, 0.5))
    plt.annotate(
        row["GO_label"],
        xy=(row["clip_enrichment"], row["mean_rdc"]),
        xytext=(row["clip_enrichment"] + dx, row["mean_rdc"] + dy),
        fontsize=9,
        ha='center',  # 🔹 수평 가운데 정렬
        va='center',  # 🔹 수직 가운데 정렬
        bbox=dict(boxstyle="round,pad=0.3", edgecolor='dimgrey', facecolor='white'),
        arrowprops=dict(arrowstyle="->", color="black", lw=0.8)
    )

plt.xticks(np.arange(-2, 3, 0.5))
plt.xlabel("Enrichment level of LIN28A-bound CLIP tags (log₂)", fontsize=11)
plt.ylabel("Ribosome density change upon Lin28a knockdown (log₂)", fontsize=11)
plt.title("Gene ontology term-enrichment analysis for CLIP and ribosome profiling", fontsize=14,
          loc='left')
plt.xlim(-2, 2.5)
plt.ylim(-1.0, 1.5)
plt.grid(True)
plt.tight_layout()
plt.savefig("/Users/jeongmin/Downloads/GO_CLIP_ribosome_enrichment_FINAL_FILTERED_0612.png", dpi=300)
# plt.show()